<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **Build a Tool Calling Agent**


### Installing required libraries


In [ ]:
%%capture
%pip install pytube
%pip install youtube-transcript-api==1.1.0
%pip install langchain-community==0.3.16
%pip install langchain==0.3.23
%pip install langchain-openai==0.3.14
%pip install yt-dlp

### Importing required libraries

_It is recommended that you import all required libraries in one place (here):_


In [ ]:
import re
from pytube import YouTube
from langchain_core.tools import tool
from IPython.display import display, JSON
import yt_dlp
from typing import List, Dict
from langchain_core.messages import HumanMessage
from langchain_core.messages import ToolMessage
import json

# Suppress warnings
import warnings
warnings.filterwarnings("ignore")

# Suppress pytube errors
import logging
pytube_logger = logging.getLogger('pytube')
pytube_logger.setLevel(logging.ERROR)

# Suppress yt-dlp warnings
yt_dpl_logger = logging.getLogger('yt_dlp')
yt_dpl_logger.setLevel(logging.ERROR)

Let's initialize the language model that will power your tool calling capabilities. This code sets up a GPT-4o-mini model using the OpenAI provider through LangChain's interface, which you'll use to process queries and decide which tools to call.


In [ ]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-4o-mini", model_provider="openai")

# Tools


In [ ]:
@tool
def extract_video_id(url: str) -> str:
    """
    Extracts the 11-character YouTube video ID from a URL.

    Args:
        url (str): A YouTube URL containing a video ID.

    Returns:
        str: Extracted video ID or error message if parsing fails.
    """

    # Regex pattern to match video IDs
    pattern = r'(?:v=|be/|embed/)([a-zA-Z0-9_-]{11})'
    match = re.search(pattern, url)
    return match.group(1) if match else "Error: Invalid YouTube URL"

## Tool list 



In [ ]:
tools = []
tools.append(extract_video_id)

### Defining transcript fetching tool

This tool uses the `YouTubeTranscriptApi` library to retrieve the captions or subtitles from a video. You'll be taking the video ID (which can be extracted using your previous tool) and an optional language parameter. The function attempts to get the transcript and joins all text segments into a continuous string, or returns an error message if the transcript can't be retrieved.


In [ ]:
from youtube_transcript_api import YouTubeTranscriptApi


@tool
def fetch_transcript(video_id: str, language: str = "en") -> str:
    """
    Fetches the transcript of a YouTube video.

    Args:
        video_id (str): The YouTube video ID (e.g., "dQw4w9WgXcQ").
        language (str): Language code for the transcript (e.g., "en", "es").

    Returns:
        str: The transcript text or an error message.
    """

    try:
        ytt_api = YouTubeTranscriptApi()
        transcript = ytt_api.fetch(video_id, languages=[language])
        return " ".join([snippet.text for snippet in transcript.snippets])
    except Exception as e:
        return f"Error: {str(e)}"

In [ ]:
tools.append(fetch_transcript)

### Defining YouTube search tool

This tool uses the `Search` class from the PyTube library to perform searches on YouTube. When given a search term, it returns a list of matching videos with each video represented as a dictionary containing the title, video ID, and a shortened URL. This tool will be helpful for discovering relevant videos when you don't already have a specific URL in mind.


In [ ]:
from pytube import Search
from langchain.tools import tool
from typing import List, Dict

@tool
def search_youtube(query: str) -> List[Dict[str, str]]:
    """
    Search YouTube for videos matching the query.

    Args:
        query (str): The search term to look for on YouTube

    Returns:
        List of dictionaries containing video titles and IDs in format:
        [{'title': 'Video Title', 'video_id': 'abc123'}, ...]
        Returns error message if search fails
    """
    try:
        s = Search(query)
        return [
            {
                "title": yt.title,
                "video_id": yt.video_id,
                "url": f"https://youtu.be/{yt.video_id}"
            }
            for yt in s.results
        ]
    except Exception as e:
        return f"Error: {str(e)}"

In [ ]:
tools.append(search_youtube)

### Defining metadata extraction tool

This tool takes a YouTube URL and returns comprehensive information about the video, including its title, view count, duration, channel name, like count, comment count, and any chapter markers.


In [ ]:
@tool
def get_full_metadata(url: str) -> dict:
    """Extract metadata given a YouTube URL, including title, views, duration, channel, likes, comments, and chapters."""
    with yt_dlp.YoutubeDL({'quiet': True, 'logger': yt_dpl_logger}) as ydl:
        info = ydl.extract_info(url, download=False)
        return {
            'title': info.get('title'),
            'views': info.get('view_count'),
            'duration': info.get('duration'),
            'channel': info.get('uploader'),
            'likes': info.get('like_count'),
            'comments': info.get('comment_count'),
            'chapters': info.get('chapters', [])
        }

In [ ]:
tools.append(get_full_metadata)

### Defining trending videos tool

This tool uses `yt-dlp` to access YouTube's trending feed based on a provided country code (like "US" for the United States or "IN" for India). It collects important information about each trending video, including the title, video ID, URL, channel name, duration, and view count.


In [ ]:
from langchain.tools import tool

from typing import List, Dict

@tool
def get_trending_videos(region_code: str) -> List[Dict]:
    """
    Fetches currently trending YouTube videos for a specific region.

    Args:
        region_code (str): 2-letter country code (e.g., "US", "IN", "GB")

    Returns:
        List of dictionaries with video details: title, video_id, channel, view_count, duration
    """
    ydl_opts = {
        'geo_bypass_country': region_code.upper(),
        'extract_flat': True,
        'quiet': True,
        'force_generic_extractor': True,
        'logger': yt_dpl_logger
    }

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(
                'https://www.youtube.com/feed/trending',
                download=False
            )

            trending_videos = []
            for entry in info['entries']:
                video_data = {
                    'title': entry.get('title', 'N/A'),
                    'video_id': entry.get('id', 'N/A'),
                    'url': entry.get('url', 'N/A'),
                    'channel': entry.get('uploader', 'N/A'),
                    'duration': entry.get('duration', 0),
                    'view_count': entry.get('view_count', 0)
                }
                trending_videos.append(video_data)

            return trending_videos[:25]  # Return top 25 trending videos

    except Exception as e:
        return [{'error': f"Failed to fetch trending videos: {str(e)}"}]

In [ ]:
tools.append(get_trending_videos)

### Defining thumbnail retrieval tool

This tool uses `yt-dlp` to retrieve information about the various thumbnail images that YouTube generates for videos at different resolutions. For each thumbnail, collect its URL, width, height, and formatted resolution.


In [ ]:
@tool
def get_thumbnails(url: str) -> List[Dict]:
    """
    Get available thumbnails for a YouTube video using its URL.

    Args:
        url (str): YouTube video URL (any format)

    Returns:
        List of dictionaries with thumbnail URLs and resolutions in YouTube's native order
    """

    try:
        with yt_dlp.YoutubeDL({'quiet': True, 'logger': yt_dpl_logger}) as ydl:
            info = ydl.extract_info(url, download=False)

            thumbnails = []
            for t in info.get('thumbnails', []):
                if 'url' in t:
                    thumbnails.append({
                        "url": t['url'],
                        "width": t.get('width'),
                        "height": t.get('height'),
                        "resolution": f"{t.get('width', '')}x{t.get('height', '')}".strip('x')
                    })

            return thumbnails

    except Exception as e:
        return [{"error": f"Failed to get thumbnails: {str(e)}"}]

In [ ]:
tools.append(get_thumbnails)

##  Binding tools




Now, you'll bind your collection of tools to the language model. It enables the LLM to access and use your custom YouTube tools during conversations. By binding the tools, you're giving the model the ability to call these functions when it determines they're needed to fulfill a user request, making the LLM aware of your tools' capabilities and how to use them.


In [ ]:
llm_with_tools = llm.bind_tools(tools)

#### Creating a tool mapping dictionary

Now you'll create a dictionary that maps tool names to their corresponding function objects. This mapping will be useful later when you need to programmatically invoke specific tools based on their names. It allows you to easily look up and execute a tool function when you have only the tool name as a string, which will be important when processing tool calls from the language model.


In [ ]:
tool_mapping = {
    "get_thumbnails" : get_thumbnails,
    "get_trending_videos": get_trending_videos,
    "extract_video_id": extract_video_id,
    "fetch_transcript": fetch_transcript,
    "search_youtube": search_youtube,
    "get_full_metadata": get_full_metadata
}

### Automating the tool calling process


In [ ]:
# Define the processing steps
def execute_tool(tool_call):
    """Execute single tool call and return ToolMessage"""
    try:
        result = tool_mapping[tool_call["name"]].invoke(tool_call["args"])
        return ToolMessage(
            content=str(result),
            tool_call_id=tool_call["id"]
        )
    except Exception as e:
        return ToolMessage(
            content=f"Error: {str(e)}",
            tool_call_id=tool_call["id"]
        )



In [ ]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda


## Building the summarization chain


In [ ]:
summarization_chain = (
    # Start with initial query
    RunnablePassthrough.assign(
        messages=lambda x: [HumanMessage(content=x["query"])]
    )
    # First LLM call (extract video ID)
    | RunnablePassthrough.assign(
        ai_response=lambda x: llm_with_tools.invoke(x["messages"])
    )
    # Process first tool call
    | RunnablePassthrough.assign(
        tool_messages=lambda x: [
            execute_tool(tc) for tc in x["ai_response"].tool_calls
        ]
    )
    # Update message history
    | RunnablePassthrough.assign(
        messages=lambda x: x["messages"] + [x["ai_response"]] + x["tool_messages"]
    )
    # Second LLM call (fetch transcript)
    | RunnablePassthrough.assign(
        ai_response2=lambda x: llm_with_tools.invoke(x["messages"])
    )
    # Process second tool call
    | RunnablePassthrough.assign(
        tool_messages2=lambda x: [
            execute_tool(tc) for tc in x["ai_response2"].tool_calls
        ]
    )
    # Final message update
    | RunnablePassthrough.assign(
        messages=lambda x: x["messages"] + [x["ai_response2"]] + x["tool_messages2"]
    )
    # Generate final summary
    | RunnablePassthrough.assign(
        summary=lambda x: llm_with_tools.invoke(x["messages"]).content
    )
    # Return just the summary text
    | RunnableLambda(lambda x: x["summary"])
)


#### Creating the initial message setup

Here you're setting up the first step of your chain that will handle the initial user query. The `RunnablePassthrough.assign` creates a component that takes an input dictionary containing a "query" and converts it into a list containing a single `HumanMessage` object.


In [ ]:
initial_setup = RunnablePassthrough.assign(
    messages=lambda x: [HumanMessage(content=x["query"])]
)

#### Defining the first LLM interaction

Here, you'll create the second step of your chain, which handles the first interaction with the language model. This component takes the formatted messages from the previous step, sends them to your tool-equipped LLM, and captures the response in a field called "ai_response."


In [ ]:
first_llm_call = RunnablePassthrough.assign(
    ai_response=lambda x: llm_with_tools.invoke(x["messages"])
)

#### Processing the first tool call

Here, you're defining the processing step that handles the LLM's first tool call. This component:
1. Executes each tool call by passing it to your `execute_tool` function, which runs the appropriate tool and returns the result as a `ToolMessage`
2. Updates the message history by combining the original messages, the LLM's response, with the tool calls, and the tool results
3. Prepares the updated conversation state for the next interaction with the LLM


In [ ]:
first_tool_processing = RunnablePassthrough.assign(
    tool_messages=lambda x: [
        execute_tool(tc) for tc in x["ai_response"].tool_calls
    ]
).assign(
    messages=lambda x: x["messages"] + [x["ai_response"]] + x["tool_messages"]
)

#### Defining the second LLM interaction

Here, you're creating the next step in your chain that handles the second interaction with the language model. This component takes the updated message history (which now includes the results from the first tool call) and sends it to the LLM again.


In [ ]:
second_llm_call = RunnablePassthrough.assign(
    ai_response2=lambda x: llm_with_tools.invoke(x["messages"])
)

#### Processing the second tool call

Here, you're defining the processing step that handles the LLM's second tool call. Similar to the first tool processing step, this component executes the tool calls (typically fetching the transcript), creates tool messages with the results, and updates the message history by combining everything for the final summarization step.


In [ ]:
second_tool_processing = RunnablePassthrough.assign(
    tool_messages2=lambda x: [
        execute_tool(tc) for tc in x["ai_response2"].tool_calls
    ]
).assign(
    messages=lambda x: x["messages"] + [x["ai_response2"]] + x["tool_messages2"]
)

#### Generating the final summary

Here, you're defining the final step that produces the summary of the YouTube video. This component:
1. Takes the complete message history (which now contains the original query, tool calls, and tool results)
2. Invokes the LLM one last time to generate a summary
3. Extracts just the content field from the LLM's response
4. Uses a RunnableLambda to return only the summary text as the final output


In [ ]:
final_summary = RunnablePassthrough.assign(
    summary=lambda x: llm_with_tools.invoke(x["messages"]).content
) | RunnableLambda(lambda x: x["summary"])

#### Assembling the complete chain

Now, you're combining all the individual components you've defined into a single cohesive chain. By piping each step to the next, you'll create a workflow that:
1. Formats the initial query
2. Gets the first LLM response (video ID extraction)
3. Processes the first tool call
4. Gets the second LLM response (transcript request)
5. Processes the second tool call
6. Generates the final summary


In [ ]:
chain = (
    initial_setup
    | first_llm_call
    | first_tool_processing
    | second_llm_call
    | second_tool_processing
    | final_summary
)

## Recursive chain flow


In [ ]:
from langchain_core.runnables import RunnableBranch, RunnableLambda
from langchain_core.messages import HumanMessage, ToolMessage
import json

def execute_tool(tool_call):
    """Execute single tool call and return ToolMessage"""
    try:
        result = tool_mapping[tool_call["name"]].invoke(tool_call["args"])
        content = json.dumps(result) if isinstance(result, (dict, list)) else str(result)
    except Exception as e:
        content = f"Error: {str(e)}"

    return ToolMessage(
        content=content,
        tool_call_id=tool_call["id"]
    )

#### Defining the core processing logic

In [ ]:
def process_tool_calls(messages):
    """Recursive tool call processor"""
    last_message = messages[-1]

    # Execute all tool calls in parallel
    tool_messages = [
        execute_tool(tc)
        for tc in getattr(last_message, 'tool_calls', [])
    ]

    # Add tool responses to message history
    updated_messages = messages + tool_messages

    # Get next LLM response
    next_ai_response = llm_with_tools.invoke(updated_messages)

    return updated_messages + [next_ai_response]

#### Creating the recursive stopping condition

In [ ]:
def should_continue(messages):
    """Check if you need another iteration"""
    last_message = messages[-1]
    return bool(getattr(last_message, 'tool_calls', None))


#### Implementing the recursive function


In [ ]:
def _recursive_chain(messages):
    """Recursively process tool calls until completion"""
    if should_continue(messages):
        new_messages = process_tool_calls(messages)
        return _recursive_chain(new_messages)
    return messages

recursive_chain = RunnableLambda(_recursive_chain)

#### Building the complete universal chain


In [ ]:
universal_chain = (
    RunnableLambda(lambda x: [HumanMessage(content=x["query"])])
    | RunnableLambda(lambda messages: messages + [llm_with_tools.invoke(messages)])
    | recursive_chain
)

# Exercise


### Exercise 1: Try a different video with a Youtube link


In [ ]:
video_url = "https://www.youtube.com/watch?v=m6G8k9dQ8S8"

### Exercise 2: Extract the video ID


In [ ]:
video_id = extract_video_id.run(video_url)
print(f"Extracted video ID: {video_id}")

### Exercise 3: Collect all necessary data about the video in one go


In [ ]:
video_metadata = get_full_metadata.run(video_url)
print(f"Retrieved metadata for: {video_metadata['title']}")

### Exercise 4: Get video transcript


In [ ]:
transcript = fetch_transcript.run(video_url)
print(f"Retrieved transcript with {len(transcript)} characters")

### Exercise 5: Get video thumbnails


In [ ]:
thumbnails = get_thumbnails.run(video_url)
print(f"Retrieved {len(thumbnails)} thumbnails")

### Let's have a comprehensive prompt to be passed to LLM to generate a summary


In [ ]:
prompt = f"""
Please analyze this YouTube video and provide a comprehensive summary.

VIDEO TITLE: {video_metadata['title']}
CHANNEL: {video_metadata['channel']}
VIEWS: {video_metadata['views']}
DURATION: {video_metadata['duration']} seconds
LIKES: {video_metadata['likes']}

TRANSCRIPT EXCERPT:
{transcript[:3000]}... (transcript truncated for brevity)

Based on this information, please provide:
1. A concise summary of the video content (3-5 bullet points)
2. The main topics or themes discussed
3. The intended audience for this content
4. A brief analysis of why this video might be performing well (or not)
"""

### Exercise 6: Single LLM invocation with all the data


In [ ]:
messages = [HumanMessage(content=prompt)]
response = llm.invoke(messages)

### Exercise 7: Display the comprehensive analysis


In [ ]:
print("\n===== VIDEO ANALYSIS =====\n")
print(response.content)

Copyright © IBM Corporation. All rights reserved.
